In [1]:
## Import whatever module needed
import math, json
from collections import Counter
from datetime import datetime, timedelta
from geopandas.tools import sjoin
# from pyproj import proj
import pandas as pd
import csv
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import fiona
from fulcrum import Fulcrum

In [2]:
## Variables
apiToken = "689a52b9420360bef6af63ad889bcbf4cb34f26e1d1348b2d3a2ff3b71dfe6828d07395724415aa9"
urlBase = 'https://api.fulcrumapp.com/api/v2/'
fulcrum = Fulcrum(key=apiToken)
osTimestamp = datetime.today() - timedelta(days=4)
recordsPerPage = 5000
print(fulcrum)

In [3]:
## Data CTI
appID = "0092aa7f-a2e8-4972-8f26-46eb98b23a12" # ID of Combined Tree Invnetory
formdataSource = fulcrum.forms.find(appID)
# get number of pages
## the following divides the total number of records in chunks and downloads and assembles them. Otherwise the fulcrum API allows only downloads of 20,000 records

## count records in app
recordCountSource = fulcrum.forms.find(appID)['form']['record_count']
pagesSource = math.ceil(recordCountSource / recordsPerPage)

## get data I
if pagesSource > 1:
    for p in range(1, pagesSource + 1):
        dataPage = fulcrum.records.search(url_params={'form_id': appID, 'page': p, 'per_page': recordsPerPage})[
            'records']
        if p > 1:
            dataSource.extend(dataPage)
        else:
            dataSource = dataPage
else:
    dataSource = fulcrum.records.search(url_params={'form_id': appID, 'page': 1, 'per_page': recordsPerPage})['records']

In [4]:
## What do we have? Attributes:
type(dataSource)
cticolumns = list(dataSource[0].keys())
cticolumns

['status',
 'version',
 'id',
 'created_at',
 'updated_at',
 'client_created_at',
 'client_updated_at',
 'created_by',
 'created_by_id',
 'updated_by',
 'updated_by_id',
 'created_location',
 'updated_location',
 'created_duration',
 'updated_duration',
 'edited_duration',
 'form_id',
 'project_id',
 'record_series_id',
 'assigned_to',
 'assigned_to_id',
 'form_values',
 'latitude',
 'longitude',
 'altitude',
 'geometry',
 'speed',
 'course',
 'horizontal_accuracy',
 'vertical_accuracy',
 'system_status']

In [5]:
print(dataSource[0])

{'status': 'No action required - لا حاجة لأي إجراء', 'version': 1, 'id': 'f8f84044-06d2-4d3e-9004-9b3116cc31ec', 'created_at': '2023-07-11T11:57:01Z', 'updated_at': '2023-07-11T11:57:01Z', 'client_created_at': '2023-07-11T11:57:01Z', 'client_updated_at': '2023-07-11T11:57:01Z', 'created_by': '[BP] admin L2', 'created_by_id': 'f8161492-3026-4f3c-a770-93f8e540d4d1', 'updated_by': '[BP] admin L2', 'updated_by_id': 'f8161492-3026-4f3c-a770-93f8e540d4d1', 'created_location': None, 'updated_location': None, 'created_duration': None, 'updated_duration': None, 'edited_duration': None, 'form_id': '0092aa7f-a2e8-4972-8f26-46eb98b23a12', 'project_id': '8b0726a7-e56e-4230-874c-e55c63693caf', 'record_series_id': None, 'assigned_to': None, 'assigned_to_id': None, 'form_values': {'2c1f': 'Campsite,Irrigation,trees watered', '6e5e': {'choice_values': ['BP inventory'], 'other_values': []}, '3fee': {'choice_values': ['Acacia sp. - السنط'], 'other_values': []}, '5cf5': [{'photo_id': '055e3eac-2472-42e5-9

In [6]:
print(dataSource[0]["form_values"])
status_values = {record["id"] for record in dataSource if "id" in record}
len(status_values)

{'2c1f': 'Campsite,Irrigation,trees watered', '6e5e': {'choice_values': ['BP inventory'], 'other_values': []}, '3fee': {'choice_values': ['Acacia sp. - السنط'], 'other_values': []}, '5cf5': [{'photo_id': '055e3eac-2472-42e5-93ed-e9ea9974b4c2', 'caption': ''}]}


302753

In [7]:
import os
file_path = r"D:\TMO\Fulcrum\MLonly.csv"
print(os.path.exists(file_path))

True


In [8]:
import pandas as pd
CTIcsv = pd.read_csv(file_path, sep=";")
print(CTIcsv.keys())
print(CTIcsv["rid"])                   # Unique record id
print(CTIcsv["rid"].nunique())         # Unique record id
print(CTIcsv["source_"])

Index(['rid', '_status', '_title', '_server_updated_at', '_project',
       '_updated_by', '_latitude', '_longitude', 'species', 'transplantable',
       'last_updated', 'height_m', 'spread_m', 'dbh_cm_', 'planting_date',
       'health', 'structure_', 'comments', 'source_', 'significant_tree',
       'reason_for_significance', 'old_status', 'id', 'tag_id_',
       'green_riyadh_project', 'marker-color', 'maintenance_', 'photos'],
      dtype='object')
0        fa2c9c41-f3b5-429e-b95b-00a011792909
1        e49b8377-bb42-459b-a474-e9e846503b5e
2        f2ec79d2-f388-46ca-8fef-2b77e6d197a2
3        f1f76a2a-0303-4a55-bd73-89c018a2e9fa
4        ecbbd956-6c37-425b-aa19-13cdc446195f
                         ...                 
48974    8ea1a2fa-7ede-4666-ac0e-4434958c4100
48975    788c40ba-79c2-4ffa-b541-594245b701f4
48976    645c4b05-ab10-4855-82f3-53e336543dcd
48977    6fc6c5f6-b0a3-4856-9b37-ea8d33cf69eb
48978    a9cb6fa3-1bbe-4e0e-b4e9-075c9f13c915
Name: rid, Length: 48979, dtype: obje

C:\Users\Jura\AppData\Local\Temp\ipykernel_14688\807536193.py:2: DtypeWarning: Columns (4,10,15,16,17,19,22) have mixed types. Specify dtype option on import or set low_memory=False.
  CTIcsv = pd.read_csv(file_path, sep=";")


In [9]:
print(dataSource[0]["form_id"])    # App id
print(dataSource[1]["form_id"])    # App id
print(dataSource[1]["id"])         # Unique record id
print(dataSource[2]["id"])         # Unique record id
print(dataSource[3]["id"])         # Unique record id

0092aa7f-a2e8-4972-8f26-46eb98b23a12
0092aa7f-a2e8-4972-8f26-46eb98b23a12
f7b3c727-e08d-4371-bd7c-9a893d9a68ff
719c30b7-578d-4b6e-97fe-ab3db8adab38
1168e1fc-fef3-4310-ae86-9b58efcef940


In [10]:
## Lookup set
cti_lookup = set(CTIcsv["rid"]) 
list(cti_lookup)[:5]

['2110517e-4e3a-4706-bd5f-965698d19ced',
 '474a7fdf-248d-4a22-896f-d39d5cdd589d',
 'a33be26c-bb42-43bb-84cb-43643ad7617c',
 'e26ac6d7-5d48-4e21-8b24-4fa3faf638bc',
 '245ffb55-1469-4eb9-a992-5df6fc66bdaf']

In [ ]:
## Add records 
MLid = "21b1350f-e906-4e96-848b-2b30cd6fd130" # App id ~ ML_PalmTrees

for record in dataSource:
    record_id = record.get("id")

    del(record["id"])

    if record_id in cti_lookup:
        # Ensure the record has the correct project ID (MLid)
        record["form_id"] = MLid

        # Check if the form_id matches the correct app ID before uploading
        if record["form_id"] == MLid:
            try:
                # Create/upload record
                created_record = fulcrum.records.create(record)
                # print(f"Record {record_id} created successfully")
            except Exception as e:
                print(f"Error creating record {record_id}: {str(e)}")
        else:
            print(f"Record {record_id} skipped due to incorrect form_id.")

In [ ]:
## Delete records 
for record in dataSource:
    record_id = record.get("id")

    # Conditional deletion (only ids that are in the lookup set are deleted)
    if record_id in cti_lookup:  
        try:
            # Delete record
            fulcrum.records.delete(record_id)
            # DELETED = fulcrum.records.delete(record_id)
            # print(f"Deleted record: {record_id}")
        except Exception as e:
            print(f"Error deleting record {record_id}: {e}")